# Nonparametric ML Models - Cumulative Lab

## Introduction

In this cumulative lab, you will apply two nonparametric models you have just learned — k-nearest neighbors and decision trees — to the forest cover dataset.

## Objectives

* Practice identifying and applying appropriate preprocessing steps
* Perform an iterative modeling process, starting from a baseline model
* Explore multiple model algorithms, and tune their hyperparameters
* Practice choosing a final model across multiple model algorithms and evaluating its performance

## Your Task: Complete an End-to-End ML Process with Nonparametric Models on the Forest Cover Dataset

![line of pine trees](images/trees.jpg)

Photo by <a href="https://unsplash.com/@michaelbenz?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Michael Benz</a> on <a href="/s/photos/forest?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a>

### Business and Data Understanding

To repeat the previous description:

> Here we will be using an adapted version of the forest cover dataset from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/covertype). Each record represents a 30 x 30 meter cell of land within Roosevelt National Forest in northern Colorado, which has been labeled as `Cover_Type` 1 for "Cottonwood/Willow" and `Cover_Type` 0 for "Ponderosa Pine". (The original dataset contained 7 cover types but we have simplified it.)

The task is to predict the `Cover_Type` based on the available cartographic variables:

In [1]:
# Run this cell without changes
import pandas as pd

df = pd.read_csv('data/forest_cover.csv')
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Cover_Type
0,2553,235,17,351,95,780,188,253,199,1410,...,0,0,0,0,0,0,0,0,0,0
1,2011,344,17,313,29,404,183,211,164,300,...,0,0,0,0,0,0,0,0,0,0
2,2022,24,13,391,42,509,212,212,134,421,...,0,0,0,0,0,0,0,0,0,0
3,2038,50,17,408,71,474,226,200,102,283,...,0,0,0,0,0,0,0,0,0,0
4,2018,341,27,351,34,390,152,188,168,190,...,0,0,0,0,0,0,0,0,0,0


> As you can see, we have over 38,000 rows, each with 52 feature columns and 1 target column:

> * `Elevation`: Elevation in meters
> * `Aspect`: Aspect in degrees azimuth
> * `Slope`: Slope in degrees
> * `Horizontal_Distance_To_Hydrology`: Horizontal dist to nearest surface water features in meters
> * `Vertical_Distance_To_Hydrology`: Vertical dist to nearest surface water features in meters
> * `Horizontal_Distance_To_Roadways`: Horizontal dist to nearest roadway in meters
> * `Hillshade_9am`: Hillshade index at 9am, summer solstice
> * `Hillshade_Noon`: Hillshade index at noon, summer solstice
> * `Hillshade_3pm`: Hillshade index at 3pm, summer solstice
> * `Horizontal_Distance_To_Fire_Points`: Horizontal dist to nearest wildfire ignition points, meters
> * `Wilderness_Area_x`: Wilderness area designation (3 columns)
> * `Soil_Type_x`: Soil Type designation (39 columns)
> * `Cover_Type`: 1 for cottonwood/willow, 0 for ponderosa pine

This is also an imbalanced dataset, since cottonwood/willow trees are relatively rare in this forest:

In [2]:
# Run this cell without changes
print("Raw Counts")
print(df["Cover_Type"].value_counts())
print()
print("Percentages")
print(df["Cover_Type"].value_counts(normalize=True))

Raw Counts
0    35754
1     2747
Name: Cover_Type, dtype: int64

Percentages
0    0.928651
1    0.071349
Name: Cover_Type, dtype: float64


Thus, a baseline model that always chose the majority class would have an accuracy of over 92%. Therefore we will want to report additional metrics at the end.

### Previous Best Model

In a previous lab, we used SMOTE to create additional synthetic data, then tuned the hyperparameters of a logistic regression model to get the following final model metrics:

* **Log loss:** 0.13031294393913376
* **Accuracy:** 0.9456679825472678
* **Precision:** 0.6659919028340081
* **Recall:** 0.47889374090247455

In this lab, you will try to beat those scores using more-complex, nonparametric models.

### Modeling

Although you may be aware of some additional model algorithms available from scikit-learn, for this lab you will be focusing on two of them: k-nearest neighbors and decision trees. Here are some reminders about these models:

#### kNN - [documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

This algorithm — unlike linear models or tree-based models — does not emphasize learning the relationship between the features and the target. Instead, for a given test record, it finds the most similar records in the training set and returns an average of their target values.

* **Training speed:** Fast. In theory it's just saving the training data for later, although the scikit-learn implementation has some additional logic "under the hood" to make prediction faster.
* **Prediction speed:** Very slow. The model has to look at every record in the training set to find the k closest to the new record.
* **Requires scaling:** Yes. The algorithm to find the nearest records is distance-based, so it matters that distances are all on the same scale.
* **Key hyperparameters:** `n_neighbors` (how many nearest neighbors to find; too few neighbors leads to overfitting, too many leads to underfitting), `p` and `metric` (what kind of distance to use in defining "nearest" neighbors)

#### Decision Trees - [documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

Similar to linear models (and unlike kNN), this algorithm emphasizes learning the relationship between the features and the target. However, unlike a linear model that tries to find linear relationships between each of the features and the target, decision trees look for ways to split the data based on features to decrease the entropy of the target in each split.

* **Training speed:** Slow. The model is considering splits based on as many as all of the available features, and it can split on the same feature multiple times. This requires exponential computational time that increases based on the number of columns as well as the number of rows.
* **Prediction speed:** Medium fast. Producing a prediction with a decision tree means applying several conditional statements, which is slower than something like logistic regression but faster than kNN.
* **Requires scaling:** No. This model is not distance-based. You also can use a `LabelEncoder` rather than `OneHotEncoder` for categorical data, since this algorithm doesn't necessarily assume that the distance between `1` and `2` is the same as the distance between `2` and `3`.
* **Key hyperparameters:** Many features relating to "pruning" the tree. By default they are set so the tree can overfit, and by setting them higher or lower (depending on the hyperparameter) you can reduce overfitting, but too much will lead to underfitting. These are: `max_depth`, `min_samples_split`, `min_samples_leaf`, `min_weight_fraction_leaf`, `max_features`, `max_leaf_nodes`, and `min_impurity_decrease`. You can also try changing the `criterion` to "entropy" or the `splitter` to "random" if you want to change the splitting logic.

### Requirements

#### 1. Prepare the Data for Modeling

#### 2. Build a Baseline kNN Model

#### 3. Build Iterative Models to Find the Best kNN Model

#### 4. Build a Baseline Decision Tree Model

#### 5. Build Iterative Models to Find the Best Decision Tree Model

#### 6. Choose and Evaluate an Overall Best Model

## 1. Prepare the Data for Modeling

The target is `Cover_Type`. In the cell below, split `df` into `X` and `y`, then perform a train-test split with `random_state=42` and `stratify=y` to create variables with the standard `X_train`, `X_test`, `y_train`, `y_test` names.

Include the relevant imports as you go.

In [3]:
# Your code here
from sklearn.model_selection import train_test_split
X = df.drop('Cover_Type', axis=1)
y = df.Cover_Type

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

Now, instantiate a `StandardScaler`, fit it on `X_train`, and create new variables `X_train_scaled` and `X_test_scaled` containing values transformed with the scaler.

In [4]:
# Your code here
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

The following code checks that everything is set up correctly:

In [5]:
# Run this cell without changes

# Checking that df was separated into correct X and y
assert type(X) == pd.DataFrame and X.shape == (38501, 52)
assert type(y) == pd.Series and y.shape == (38501,)

# Checking the train-test split
assert type(X_train) == pd.DataFrame and X_train.shape == (28875, 52)
assert type(X_test) == pd.DataFrame and X_test.shape == (9626, 52)
assert type(y_train) == pd.Series and y_train.shape == (28875,)
assert type(y_test) == pd.Series and y_test.shape == (9626,)

# Checking the scaling
assert X_train_scaled.shape == X_train.shape
assert round(X_train_scaled[0][0], 3) == -0.636
assert X_test_scaled.shape == X_test.shape
assert round(X_test_scaled[0][0], 3) == -1.370

## 2. Build a Baseline kNN Model

Build a scikit-learn kNN model with default hyperparameters. Then use `cross_val_score` with `scoring="neg_log_loss"` to find the mean log loss for this model (passing in `X_train_scaled` and `y_train` to `cross_val_score`). You'll need to find the mean of the cross-validated scores, and negate the value (either put a `-` at the beginning or multiply by `-1`) so that your answer is a log loss rather than a negative log loss.

Call the resulting score `knn_baseline_log_loss`.

Your code might take a minute or more to run.

In [9]:
# Replace None with appropriate code

# Relevant imports
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
# Creating the model
knn_baseline_model = KNeighborsClassifier()

# Perform cross-validation
knn_baseline_log_loss = np.mean(cross_val_score(knn_baseline_model, X_train_scaled, y_train, scoring='neg_log_loss'))

-knn_baseline_log_loss

0.1255288892455634

Our best logistic regression model had a log loss of 0.13031294393913376

Is this model better? Compare it in terms of metrics and speed.

In [ ]:
# Replace None with appropriate text
"""
The log loss for KNN is lower than that of logistic model, and lower log loss means better. 

"""

## 3. Build Iterative Models to Find the Best kNN Model

Build and evaluate at least two more kNN models to find the best one. Explain why you are changing the hyperparameters you are changing as you go. These models will be *slow* to run, so be thinking about what you might try next as you run them.

In [16]:

#default number of neighbors is 5, so i increase the number since there is lots records in the data
loss = []
for idx, i in enumerate([10,15,20,25,30,40,45,50]):

    knn_model1 = KNeighborsClassifier(n_neighbors=i)

# Perform cross-validation
    knn_model1_log_loss = np.mean(cross_val_score(knn_model1, X_train_scaled, y_train, scoring='neg_log_loss'))

    loss.append(-knn_model1_log_loss)
    print('Log loss for',i,' is ',loss[idx])

Log loss for 10  is  0.07371926754949004
Log loss for 15  is  0.06722997851589525
Log loss for 20  is  0.0642543665324357
Log loss for 25  is  0.06425722742416393
Log loss for 30  is  0.06806268101991848
Log loss for 40  is  0.07416964356915844
Log loss for 45  is  0.07597030403878771
Log loss for 50  is  0.078613760394212


In [ ]:
'''
The log loss decreases as neigbours increase upto 20 then starts increasing again.It's best at 20 neighbours
'''

In [17]:

# changing from default euclidean to manhattan distance and default n_neighbors
knn_model2 = KNeighborsClassifier(metric="manhattan")

# Perform cross-validation
knn_model2_log_loss = np.mean(cross_val_score(knn_model2, X_train_scaled, y_train, scoring='neg_log_loss'))

-knn_model2_log_loss

0.1144531688354272

In [ ]:
'''
Manhattan distance with default n_neigbours does better than the default euclidean distance
'''

In [18]:
# manhattan distance plus different number of neighbors
loss = []
for idx, i in enumerate([10,15,20,25,30,40,45,50]):

    knn_model1 = KNeighborsClassifier(n_neighbors=i,metric="manhattan")

# Perform cross-validation
    knn_model1_log_loss = np.mean(cross_val_score(knn_model1, X_train_scaled, y_train, scoring='neg_log_loss'))

    loss.append(-knn_model1_log_loss)
    print('Log loss for',i,' is ',loss[idx])

Log loss for 10  is  0.06254007184711532
Log loss for 15  is  0.061714464254211454
Log loss for 20  is  0.06209962711650543
Log loss for 25  is  0.06519186918054885
Log loss for 30  is  0.06761000224740496
Log loss for 40  is  0.07093116730455953
Log loss for 45  is  0.07359365820106689
Log loss for 50  is  0.07621145166565102


In [ ]:
'''
With manhattan distance, the log loss are better for same neighbors than euclidean distance,with 15
neighbors taking the day. So the KNN with 15 neighbors and manhattan distance is best
'''

## 4. Build a Baseline Decision Tree Model

Now that you have chosen your best kNN model, start investigating decision tree models. First, build and evaluate a baseline decision tree model, using default hyperparameters (with the exception of `random_state=42` for reproducibility).

(Use cross-validated log loss, just like with the previous models.)

In [19]:
# Your code here
from sklearn.tree import DecisionTreeClassifier

dt_baseline_model = DecisionTreeClassifier(random_state=42)

# Perform cross-validation
dt_baseline_log_loss = np.mean(cross_val_score(dt_baseline_model, X_train_scaled, y_train, scoring='neg_log_loss'))

-dt_baseline_log_loss

0.7057351605151588

Interpret this score. How does this compare to the log loss from our best logistic regression and best kNN models? Any guesses about why?

In [ ]:
# Replace None with appropriate text
"""
Its way worse than the best KNN and best logistic regression. No tuning had been done to the 
hyperparameters of this tree, so it could be overfitting
"""

## 5. Build Iterative Models to Find the Best Decision Tree Model

Build and evaluate at least two more decision tree models to find the best one. Explain why you are changing the hyperparameters you are changing as you go.

In [24]:
# All the hyperparameters I will use will be geared towards reducing overfitting
# 1. Max-depth
loss = []
for idx, i in enumerate([2,3,4,5,6,7,8,9]):

    dt_model1 = DecisionTreeClassifier(random_state=42,max_depth=i)

# Perform cross-validation
    dt_model1_log_loss = np.mean(cross_val_score(dt_model1, X_train_scaled, y_train, scoring='neg_log_loss'))

    loss.append(-dt_model1_log_loss)
    print('Log loss for a max depth of ',i,' is ',loss[idx])

Log loss for a max depth of  2  is  0.17110532803415182
Log loss for a max depth of  3  is  0.14950086433739623
Log loss for a max depth of  4  is  0.13012531033354707
Log loss for a max depth of  5  is  0.12002551991685706
Log loss for a max depth of  6  is  0.11365679731495566
Log loss for a max depth of  7  is  0.12915809655292862
Log loss for a max depth of  8  is  0.1843390271598817
Log loss for a max depth of  9  is  0.26191104666775517


In [ ]:
'''Max depth of 6 looks optimum in this case'''

In [33]:
#2. Min_sample_splits- the number of samples needed on internal nodes for a split to happen
loss = []
for idx, i in enumerate([100,150,200,250,300,350,400,450,500]):

    dt_model1 = DecisionTreeClassifier(random_state=42,min_samples_split=i)

# Perform cross-validation
    dt_model1_log_loss = np.mean(cross_val_score(dt_model1, X_train_scaled, y_train, scoring='neg_log_loss'))

    loss.append(-dt_model1_log_loss)
    print('Log loss for min sample split of ',i,' is ',loss[idx])

Log loss for min sample split of  100  is  0.16826341942736656
Log loss for min sample split of  150  is  0.14940297156091173
Log loss for min sample split of  200  is  0.14452346599361215
Log loss for min sample split of  250  is  0.13898962262778075
Log loss for min sample split of  300  is  0.12869629416670791
Log loss for min sample split of  350  is  0.12404846115972341
Log loss for min sample split of  400  is  0.11602063956676012
Log loss for min sample split of  450  is  0.11966300985289861
Log loss for min sample split of  500  is  0.12047624951111977


In [ ]:
'''Optimum min sample splits is about 400'''

In [34]:
loss = []
for idx, i in enumerate([100,150,200,250,300,350,400,450,500]):

    dt_model1 = DecisionTreeClassifier(random_state=42,min_samples_leaf=i)

# Perform cross-validation
    dt_model1_log_loss = np.mean(cross_val_score(dt_model1, X_train_scaled, y_train, scoring='neg_log_loss'))

    loss.append(-dt_model1_log_loss)
    print('Log loss for min sample split of ',i,' is ',loss[idx])

Log loss for min sample split of  100  is  0.11894015917756935
Log loss for min sample split of  150  is  0.11277684302498606
Log loss for min sample split of  200  is  0.10971449286975354
Log loss for min sample split of  250  is  0.1144931139252577
Log loss for min sample split of  300  is  0.1210069857263327
Log loss for min sample split of  350  is  0.12828434443242345
Log loss for min sample split of  400  is  0.12997080850714532
Log loss for min sample split of  450  is  0.13575954676586122
Log loss for min sample split of  500  is  0.141792722564956


In [ ]:
'''Optimum Minimum sample leaf is about 200 at a log loss of 0.109 and the loss is better 
than the 400 minimum sample split whose log is 0.116'''

In [37]:
dt_final_model = DecisionTreeClassifier(random_state=42, min_samples_leaf=200)

# Perform cross-validation
dt_final_log_loss = np.mean(cross_val_score(dt_final_model, X_train_scaled, y_train, scoring='neg_log_loss'))

-dt_final_log_loss

0.10971449286975354

In [ ]:
'''This final decision tree model is worse than the baseline and also worse than the KNN, although decision tree is faster'''

## 6. Choose and Evaluate an Overall Best Model

Which model had the best performance? What type of model was it?

Instantiate a variable `final_model` using your best model with the best hyperparameters.

In [38]:
# Replace None with appropriate code
final_model = KNeighborsClassifier(n_neighbors=15, metric="manhattan")

# Fit the model on the full training data
# (scaled or unscaled depending on the model)
final_model.fit(X_train_scaled,y_train)

KNeighborsClassifier(metric='manhattan', n_neighbors=15)

Now, evaluate the log loss, accuracy, precision, and recall. This code is mostly filled in for you, but you need to replace `None` with either `X_test` or `X_test_scaled` depending on the model you chose.

In [40]:
# Replace None with appropriate code
from sklearn.metrics import accuracy_score, precision_score, recall_score,log_loss

preds = final_model.predict(X_test_scaled)
probs = final_model.predict_proba(X_test_scaled)

print("log loss: ", log_loss(y_test, probs))
print("accuracy: ", accuracy_score(y_test, preds))
print("precision:", precision_score(y_test, preds))
print("recall:   ", recall_score(y_test, preds))

log loss:  0.06478510440720736
accuracy:  0.9791190525659672
precision: 0.902317880794702
recall:    0.7933042212518195


Interpret your model performance. How would it perform on different kinds of tasks? How much better is it than a "dummy" model that always chooses the majority class, or the logistic regression described at the start of the lab?

In [ ]:
# Replace None with appropriate text
"""
With an accuracy of 97% in this final model, its better than a dummy model that chooses the majority class,
whose accuracy as seen earlier was 92%


If the model predicts that a tree is in class 1, there is a 90% chance that the tree belongs to that class and
10% chance that its not.This is better than the 67% from logistic regression


Recall of 79%, so if a tree is in class 1, there is a 79% chance that it is correctly identified to be in class 1
and 21% chance of being incorrectly classified. With the imbalance in this dataset with class 1 numbers being quite low, 
there needs to be a higher recall.False negatives need to be reduced.
"""

## Conclusion

In this lab, you practiced the end-to-end machine learning process with multiple model algorithms, including tuning the hyperparameters for those different algorithms. You saw how nonparametric models can be more flexible than linear models, potentially leading to overfitting but also potentially reducing underfitting by being able to learn non-linear relationships between variables. You also likely saw how there can be a tradeoff between speed and performance, with good metrics correlating with slow speeds.